In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import dash

In [3]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

schema = Schema(beer_id=STORED(),
                brewery=TEXT(stored=True),
                beer=TEXT(stored=True),
                group=KEYWORD)

In [4]:
df = pd.read_csv('Beer_Dataset',encoding='utf-8')

In [5]:
df[df.beer=='Dragon\'s Milk'].mean()

rDev          -2.245122
rating         3.938659
beer_id    77623.817073
dtype: float64

In [6]:
import os.path
from whoosh.index import create_in

if not os.path.exists("index"):
    os.mkdir("index")
ix = create_in("index", schema)

In [7]:
from whoosh.index import open_dir

ix = open_dir("index")

In [8]:
writer = ix.writer()

writer.add_document(beer_id = df.iloc[0].beer_id, beer_name = df.iloc[0].beer, brewery= df.iloc[0].brewery)

In [9]:
docs = df[['brewery','beer_id','beer','group']].groupby(['beer_id','brewery','beer','group']).count().reset_index()

In [10]:
for t in range(len(docs)):
    writer.add_document(brewery = docs.brewery[t], beer = docs.beer[t], beer_id = docs.beer_id[t], group = docs.group[t])

In [11]:
beers = list(set(df.beer))
brews = list(set(df.brewery))

In [12]:
for r in range(len(brews)):
    writer.add_document(brewery = brews[r])

In [13]:
for k in range(len(beers)):
    writer.add_document(beer = beers[k])

In [14]:
writer.commit()

In [15]:
with ix.searcher() as searcher:
    from whoosh.query import *
    myquery = And([Term("beer", u"Sculpin"),Term("brewery",u'Ballast')])
    results = searcher.search(myquery)
    print results

<Top 0 Results for And([Term('beer', u'Sculpin'), Term('brewery', u'Ballast')]) runtime=0.000218803378908>


In [16]:
df[df.beer==u'Moose Drool Brown Ale']
df[df.beer.str.contains('dragonmilk')]

,beer,brewery,rDev,rating,username,beer_id,type,group


In [17]:
#search breweries
from whoosh.qparser import QueryParser, MultifieldParser

qp = MultifieldParser(["brewery", "beer"], schema=ix.schema)
#qp = QueryParser(["brewery","beer"], schema=ix.schema)
q = qp.parse('sculpin')

with ix.searcher() as s:
    results = s.search(q)
    ids = []; liner = []
    for x in range(5):
        a = results[x]['beer']
        b = results[x]['brewery']
        c = str(results[x]['beer_id'])
        new_line = str(a + ' ' + b)
        ids.append(c)
        liner.append(new_line)
        
print liner, ids

['Coffee Sculpin Ballast Point Brewing Company', 'Grapefruit Sculpin Ballast Point Brewing Company', 'Habanero Sculpin Ballast Point Brewing Company', 'Pineapple Sculpin Ballast Point Brewing Company', 'Sculpin IPA Ballast Point Brewing Company'] ['8503', '8514', '8518', '8560', '8579']


In [18]:
[None]

[None]

In [19]:
qp = MultifieldParser(["brewery","beer"], schema=ix.schema)
#q = qp.parse(entry)
q = qp.parse(u'fat tire amber')
with ix.searcher() as s:
    results = s.search(q)
    resulte = [results[r] for r in range(len(results)) if len(results[r])==3]
    liner = []
    ids = []
    for x in range(len(resulte)):
        try:
            be = resulte[x]['beer']
            br = resulte[x]['brewery']
            bi = resulte[x]['beer_id']
            new_line = str(be + ', ' + br)
            ids.append(bi)
            liner.append(new_line)
        except:
            print results[x]

    lines = []
    for d in range(len(resulte)):
        ent = {'label':liner[d],'value':ids[d]}
        lines.append(ent)
    print lines

[{'value': 76970, 'label': 'Fat Tire Amber Ale, New Belgium Brewing'}]


In [20]:
a = ['b','c']
print '{}'.format(a)

['b', 'c']


In [21]:
line1 = str(a + ' ' + b)
line2 = str(d + ' ' + e)
line3 = str(g + ' ' + h)
lines = (line1,line2,line3)
print '\n'.join(lines)

TypeError: can only concatenate list (not "str") to list

In [ ]:
#search beers
qp = QueryParser("beer", schema=ix.schema)
p = qp.parse(u"Moose")

with ix.searcher() as s:
    results = s.search(p)
    print results[::]